In [ ]:
pip install --upgrade pip

In [2]:
pip install jupyter openai wheel pyobjc playsound pvrecorder

  Using cached jupyter-1.1.1-py2.py3-none-any.whl.metadata (2.0 kB)
  Using cached playsound-1.3.0.tar.gz (7.7 kB)
  Preparing metadata (setup.py) ... done
  Using cached comm-0.2.2-py3-none-any.whl.metadata (3.7 kB)
  Using cached jinja2-3.1.4-py3-none-any.whl.metadata (2.6 kB)
  Using cached MarkupSafe-2.1.5-cp311-cp311-macosx_10_9_x86_64.whl.metadata (3.0 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 4.2 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.6/11.6 MB 7.0 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.0/5.0 MB 6.1 MB/s eta 0:00:0000:0100:01
Using cached comm-0.2.2-py3-none-any.whl (7.2 kB)
Using cached jinja2-3.1.4-py3-none-any.whl (133 kB)
Using cached MarkupSafe-2.1.5-cp311-cp311-macosx_10_9_x86_64.whl (14 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 5.2 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.6/9.6 MB 3.4 MB/s eta 0:00:00a 0:00:01
  Created wheel for playsound: 

In [3]:
import wave, struct, os
from openai import OpenAI
from pvrecorder import PvRecorder
from playsound import playsound
from IPython.display import Image, display

 # silence deprecation warning
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)

In [8]:
client = OpenAI()

In [5]:
	
class Chatbot:
    def __init__(self, client):
        self.client = client
        self.context = [
            {"role": "system", "content": "You are a serious assistant, der immer präzise recherchiert und hilfsbereit antwortet."},
        ]

    def chat(self, message):
        self.context.append(
            {"role": "user", "content": message}
        )
        response = self.client.chat.completions.create(
            model="gpt-4-0125-preview",
            messages=self.context
        )
        response_content = response.choices[0].message.content
        self.context.append(
            {"role": "assistant", "content": response_content}
        )
        self.show_face(response_content)
        self.print_chat()
        self.speak(response_content)

    def speak(self, message, index=0):
        speech_file_path = os.getcwd() + f"/speech_{index}.mp3"
        response = client.audio.speech.create(
            model="tts-1-hd",
            voice="shimmer",
            input=message
        )
        response.stream_to_file(speech_file_path)
        playsound(speech_file_path)

    def record_audio(self, index=0):
        recorder = PvRecorder(device_index=-1, frame_length=512)
        audio = []
        filepath = os.getcwd() + f"/recorded_{index}.mp3"
        
        try:
            recorder.start()
            print("Audio recording started ...")
            while True:
                frame = recorder.read()
                audio.extend(frame)
        except KeyboardInterrupt:
            recorder.stop()
            print("Audio recording stopped ...")
            with wave.open(filepath, 'w') as f:
                f.setparams((1, 2, 16000, 512, "NONE", "NONE"))
                f.writeframes(struct.pack("h" * len(audio), *audio))
        finally:
            recorder.delete()
            return filepath

    def transcribe(self, audio_path):
        audio_file= open(audio_path, "rb")
        transcript = client.audio.transcriptions.create(
            model="whisper-1", 
            file=audio_file
        )
        return transcript.text 

    def voicechat(self):
        recorded_filepath = self.record_audio(index=len(self.context))
        message = self.transcribe(recorded_filepath)
        self.chat(message)

    def show_face(self, message):
        response = self.client.chat.completions.create(
            # model="gpt-4-0125-preview",
            model="gpt-4-turbo",
            messages=[
                {"role": "system", "content": """
                    You are a face describing system which describes the face with a picture of proffessor Dr. Claudia Förster from FH Rosenheim (https://www.th-rosenheim.de/die-hochschule/fakultaeten/fakultaet-fuer-informatik/prof-dr-claudia-foerster) making a comprehensive, but very friendly comment.
						You receive the text the person is saying, please describe the face that would be fitting as a prompt to the stable diffusion image generation AI, DALL·E.
                """},
                {"role": "user", "content": message},
              ]
        )
        image_description = response.choices[0].message.content

        response = client.images.generate(
            model="dall-e-3",
            prompt=image_description,
            size="1024x1024",
            quality="standard",
            n=1,
        )
        
        display(Image(url=response.data[0].url))

    def print_chat(self):
        for message in self.context:
            if message["role"] == "user":
                print(f'USER: {message["content"]}')
            elif message["role"] == "assistant":
                print(f'BOT: {message["content"]}')

In [6]:
Chatbot = Chatbot(client)

In [7]:
Chatbot.voicechat()

Audio recording started ...
Audio recording stopped ...


USER: you
BOT: Ich bin hier, um Ihnen mit Informationen oder bei der Lösung von Anfragen zu helfen. Wie kann ich Ihnen heute assistieren?
